# requirements

In [ ]:
!pip install deeppavlov fasttext pymorphy2

In [ ]:
!python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [3]:
!set "KERAS_BACKEND=tensorflow"

In [4]:
import numpy as np
import re
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

# Загрузим и посмотрим на текст

In [5]:
with open('Dostoevsky.txt', 'r') as f:
    text = f.read()

print(text)

В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С — м переулке, на улицу и медленно, как бы в нерешимости, отправился к К — ну мосту.
Он благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу. И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться.
Не то чтоб он был так труслив и забит, совсем даже напротив; но с некоторого времени он был в раздражительном и напряженном состоянии, похо

In [6]:
print(len(re.split('\W+', text)))

302


# Загрузим модели и инициализируем классы для обработки текстов

In [ ]:
# Определение частей речи
from deeppavlov import build_model, configs
model = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, download=True)

# Предобученные ELMo эмбеддинги
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

# Лемматизация
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()

В качестве модели эмбедингов был взят предобученный ELMo, потому как он лучше всего подходят для задачи классификации, в условии ограниченных ресурсов. Так же ELMo можно дообучить на своих данных.
В качестве альтернативы можно было взять предобученный fasttext, который хорошо подходит для русского языка. 
При бОльших вычислительных ресурсах, имеет смысл брать предобученный bert.
Остальные эмбединги вроде tf-idf, word2vec и т.д. на мой взгляд обладают рядом минусов, особенно в условиях русского языка.

# Обьявим функции для лемматизации, определения части речи, векторизации

In [8]:
def normalize(word):
    return morph.parse(word)[0].normal_form

In [9]:
def info(text):
    """
    Возвращает информацию о тексте: 
    return (слово, лемма, часть речи)
    """
    parse = model([text])[0]
    dp_pos_info = [raw.split('\t') for raw in parse.split('\n')[:-1]]
    
    info = [(el[1], normalize(el[1]), el[2]) for el in dp_pos_info]
    return info

In [10]:
def vectorize(text):
    """Возвращает numpy вектор эмбеддинга текста."""
    tokens = re.split('\W+', text)
    return elmo([tokens])

In [11]:
info(text)

[('В', 'в', 'ADP'),
 ('начале', 'начало', 'NOUN'),
 ('июля', 'июль', 'NOUN'),
 (',', ',', 'PUNCT'),
 ('в', 'в', 'ADP'),
 ('чрезвычайно', 'чрезвычайно', 'ADV'),
 ('жаркое', 'жаркое', 'ADJ'),
 ('время', 'время', 'NOUN'),
 (',', ',', 'PUNCT'),
 ('под', 'под', 'ADP'),
 ('вечер', 'вечер', 'NOUN'),
 (',', ',', 'PUNCT'),
 ('один', 'один', 'NUM'),
 ('молодой', 'молодая', 'ADJ'),
 ('человек', 'человек', 'NOUN'),
 ('вышел', 'выйти', 'VERB'),
 ('из', 'из', 'ADP'),
 ('своей', 'свой', 'DET'),
 ('каморки', 'каморка', 'NOUN'),
 (',', ',', 'PUNCT'),
 ('которую', 'который', 'PRON'),
 ('нанимал', 'нанимать', 'VERB'),
 ('от', 'от', 'ADP'),
 ('жильцов', 'жильцов', 'NOUN'),
 ('в', 'в', 'ADP'),
 ('С', 'с', 'ADP'),
 ('—', '—', 'PUNCT'),
 ('м', 'метр', 'NOUN'),
 ('переулке', 'переулок', 'NOUN'),
 (',', ',', 'PUNCT'),
 ('на', 'на', 'ADP'),
 ('улицу', 'улица', 'NOUN'),
 ('и', 'и', 'CCONJ'),
 ('медленно', 'медленно', 'ADV'),
 (',', ',', 'PUNCT'),
 ('как', 'как', 'ADV'),
 ('бы', 'бы', 'PART'),
 ('в', 'в', 'ADP'),

In [12]:
vectorize(text)

array([[-0.07166423,  0.16918154, -0.08359444, ..., -0.03321989,
         0.03883274,  0.1067197 ]], dtype=float32)